In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler,RobustScaler
import torch
import torch.nn as nn  # Base neural network module
from torch.nn import Linear, ReLU, Sigmoid, Module  # Optional specific layers

from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

In [3]:
overview_data = pd.read_csv(r'D:\github_repo_forked\lifetime_value\house-prices-advanced-regression-techniques\train.csv')

## 检查数据：
- 每一列的类型
    - 数值型
    - 类别型
        - 类别数目
        - 类别分布
- 每一列的缺失值
    - 缺失值的比例
    - 缺失值的分布

In [4]:
def custom_missing_count(df):
    # Count NaN/None values but exclude 'NA' strings
    missing_values = pd.Series(index=df.columns, dtype=int)
    
    for col in df.columns:
        # For object (string) columns
        if df[col].dtype == 'object':
            # Count only None and NaN, exclude 'NA' strings
            missing_values[col] = pd.isna(df[col]).sum()
        else:
            # For numeric columns, use standard isnull
            missing_values[col] = df[col].isnull().sum()
    
    return missing_values

In [5]:
def data_overview(df, head_rows=5):
    """
    Provides a comprehensive overview of a DataFrame, including:
    - Shape of the DataFrame
    - First few rows (default 5)
    - Info (data types, non-null counts)
    - Basic descriptive statistics
    - Missing values (count and percentage)
    - Duplicate rows count
    - Number of unique values and the unique values for each column
    """
    print("======== DATA OVERVIEW ========")
    print(f"\n1. Shape of DataFrame: {df.shape[0]} rows, {df.shape[1]} columns\n")

    print(f"2. First {head_rows} Rows:\n")
    display(df.head(head_rows))

    print("\n3. DataFrame Info:\n")
    print(df.info())

    print("\n4. Descriptive Statistics (Numerical Columns):\n")
    print(df.describe().transpose())

    print("\n5. Missing Values Count and Percentage:\n")
    missing_values = custom_missing_count(df)

    missing_percentage = (df.isnull().sum() / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Count': missing_values,
        'Missing Percentage (%)': missing_percentage
    })
    missing_data = missing_data[missing_data["Missing Count"] > 0].sort_values(by='Missing Percentage (%)', ascending=False)
    
    if not missing_data.empty:
        display(missing_data)
    else:
        print("No missing values")

    print("\n6. Duplicate Rows Count:\n")
    duplicate_count = df.duplicated().sum()
    print(f"{duplicate_count} duplicate rows found\n" if duplicate_count > 0 else "No duplicate rows found\n")

    print("\n7. Number of Unique Values per Column:\n")
    unique_counts = df.nunique()
    unique_values = {col: df[col].unique() for col in df.columns}
    for col in unique_counts.index:
        print(f"{col}: {unique_counts[col]} unique values")
        if unique_counts[col] <= 10:  # Print unique values only if there are <= 10 unique values for readability
            print(f"   Unique Values: {unique_values[col]}\n")
        else:
            print("   Too many unique values to display\n")

    print("===============================\n")

In [6]:
data_overview(overview_data)

======== DATA OVERVIEW ========

1. Shape of DataFrame: 1460 rows, 81 columns

2. First 5 Rows:



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000



3. DataFrame Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   

,Missing Count,Missing Percentage (%)
PoolQC,1453.0,99.520548
MiscFeature,1406.0,96.301370
Alley,1369.0,93.767123
Fence,1179.0,80.753425
MasVnrType,872.0,59.726027
FireplaceQu,690.0,47.260274
LotFrontage,259.0,17.739726
GarageType,81.0,5.547945
GarageYrBlt,81.0,5.547945
GarageFinish,81.0,5.547945



6. Duplicate Rows Count:

No duplicate rows found


7. Number of Unique Values per Column:

Id: 1460 unique values
   Too many unique values to display

MSSubClass: 15 unique values
   Too many unique values to display

MSZoning: 5 unique values
   Unique Values: ['RL' 'RM' 'C (all)' 'FV' 'RH']

LotFrontage: 110 unique values
   Too many unique values to display

LotArea: 1073 unique values
   Too many unique values to display

Street: 2 unique values
   Unique Values: ['Pave' 'Grvl']

Alley: 2 unique values
   Unique Values: [nan 'Grvl' 'Pave']

LotShape: 4 unique values
   Unique Values: ['Reg' 'IR1' 'IR2' 'IR3']

LandContour: 4 unique values
   Unique Values: ['Lvl' 'Bnk' 'Low' 'HLS']

Utilities: 2 unique values
   Unique Values: ['AllPub' 'NoSeWa']

LotConfig: 5 unique values
   Unique Values: ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']

LandSlope: 3 unique values
   Unique Values: ['Gtl' 'Mod' 'Sev']

Neighborhood: 25 unique values
   Too many unique values to display

Condition1: 

In [7]:
def fill_missing_value(df):    
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('wu')
        else:
            df[col] = df[col].fillna(df[col].mean())


In [8]:
fill_missing_value(overview_data)

In [9]:
data_overview(overview_data)

======== DATA OVERVIEW ========

1. Shape of DataFrame: 1460 rows, 81 columns

2. First 5 Rows:



,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,wu,Reg,Lvl,AllPub,...,0,wu,wu,wu,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,wu,Reg,Lvl,AllPub,...,0,wu,wu,wu,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,wu,IR1,Lvl,AllPub,...,0,wu,wu,wu,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,wu,IR1,Lvl,AllPub,...,0,wu,wu,wu,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,wu,IR1,Lvl,AllPub,...,0,wu,wu,wu,0,12,2008,WD,Normal,250000



3. DataFrame Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   

In [10]:
obj = []
num = []
for colum,tp in overview_data.dtypes.to_dict().items():
    if tp == 'object':
        overview_data[colum] = overview_data[colum].astype('category')
        obj.append(colum)
    else:
        overview_data[colum] = overview_data[colum].astype('float64')
        num.append(colum)
print(obj)
print(num)
overview_data.dtypes

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '

Id                float64
MSSubClass        float64
MSZoning         category
LotFrontage       float64
LotArea           float64
                   ...   
MoSold            float64
YrSold            float64
SaleType         category
SaleCondition    category
SalePrice         float64
Length: 81, dtype: object

In [11]:
overview_data.dtypes

Id                float64
MSSubClass        float64
MSZoning         category
LotFrontage       float64
LotArea           float64
                   ...   
MoSold            float64
YrSold            float64
SaleType         category
SaleCondition    category
SalePrice         float64
Length: 81, dtype: object

In [12]:
catogorical_feature =['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
numerical_feature = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [13]:
[overview_data[key].nunique() for key in catogorical_feature]

[5,
 2,
 3,
 4,
 4,
 2,
 5,
 3,
 25,
 9,
 8,
 5,
 8,
 6,
 8,
 15,
 16,
 4,
 4,
 5,
 6,
 5,
 5,
 5,
 7,
 7,
 6,
 5,
 2,
 6,
 4,
 7,
 6,
 7,
 4,
 6,
 6,
 3,
 4,
 5,
 5,
 9,
 6]

In [14]:
overview_data[catogorical_feature]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
1,RL,Pave,wu,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
2,RL,Pave,wu,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
3,RL,Pave,wu,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,wu,wu,wu,WD,Abnorml
4,RL,Pave,wu,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
1456,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,wu,MnPrv,wu,WD,Normal
1457,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,wu,GdPrv,Shed,WD,Normal
1458,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,wu,wu,wu,WD,Normal


*** 
### 数据处理

类别数据用label进入dnn的embedding层，数值特征进行归一化

In [15]:
def preprocess_features(df):
    df_processed = df.copy()
        
    for feat in catogorical_feature:
        le = LabelEncoder()
        df_processed[feat] = df_processed[feat].astype(str)
        df_processed[feat] = le.fit_transform(df_processed[feat].fillna('0'))

    for feat in numerical_feature:
        feature_scaler = StandardScaler()
        log_transformed = np.log1p(df_processed[feat].values)
        df_processed[feat] = feature_scaler.fit_transform(log_transformed.reshape(-1, 1))
    
    return df_processed

In [16]:
def dnn_split(df):
    # Split features and target
    X = df.drop(['SalePrice', 'Id'], axis=1)    
    y = df['SalePrice']
    
    # Train test split
    x_train, x_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return x_train, x_eval, y_train, y_eval

In [17]:
overview_data[catogorical_feature]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
1,RL,Pave,wu,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
2,RL,Pave,wu,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
3,RL,Pave,wu,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,wu,wu,wu,WD,Abnorml
4,RL,Pave,wu,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,...,Attchd,RFn,TA,TA,Y,wu,wu,wu,WD,Normal
1456,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,Unf,TA,TA,Y,wu,MnPrv,wu,WD,Normal
1457,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,...,Attchd,RFn,TA,TA,Y,wu,GdPrv,Shed,WD,Normal
1458,RL,Pave,wu,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,wu,wu,wu,WD,Normal


*** 
### 模型

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def embedding_dim(x):
    return int(x**0.25) + 1

class EmbeddingLayer(nn.Module):
    def __init__(self, vocab_size):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim(vocab_size))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.embedding(x)
        x = self.flatten(x)
        return x

class DNNModel(nn.Module):
    def __init__(self, output_units, df):
        super(DNNModel, self).__init__()
        self.numeric_layers = nn.ModuleList([
            nn.Linear(1, 4) for _ in numerical_feature
        ])
        self.embedding_layers = nn.ModuleList([
            EmbeddingLayer(df[key].nunique()) for key in catogorical_feature
        ])
        # 计算每个嵌入层的维度
        numeric_dims = [4 for _ in numerical_feature]

        embedding_dims = [embedding_dim(df[feature].nunique()) for feature in catogorical_feature]
        # 计算总的嵌入维度
        total_input_dim = sum(numeric_dims) + sum(embedding_dims)

        self.deep_model = nn.Sequential(
            nn.Linear(total_input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, output_units)
        )

    def forward(self, inputs):
        embedding_inputs = inputs[:len(catogorical_feature)].long()
        numeric_inputs = inputs[len(catogorical_feature):]    
        

        # 假设我们有一个变量来记录当前切分的起始位置
        start_index = 0
        embedding_outputs = []
        for layer in self.embedding_layers:
            # 获取当前嵌入层的 embedding_dim
            embedding_dim = layer.embedding.embedding_dim
            # 从输入中取出相应大小的 tensor
            input_tensor = inputs[:, start_index:start_index + embedding_dim].long()
            # 传入嵌入层进行处理
            output = layer(input_tensor)
            embedding_outputs.append(output)
            # 更新起始位置
            start_index += embedding_dim

        numeric_start_index = 0
        numeric_outputs = []
        for layer in self.numeric_layers:
            # 这里假设每个数值特征线性层输入维度为 1
            input_tensor = numeric_inputs[:, numeric_start_index:numeric_start_index + 1].float()
            output = F.relu(layer(input_tensor))
            numeric_outputs.append(output)
            numeric_start_index += 1

            # 检查嵌入输入是否超出范围
        for i, input in enumerate(embedding_inputs):
            num_embeddings = self.embedding_layers[i].embedding.num_embeddings
            if (input < 0).any() or (input >= num_embeddings).any():
                print(f"第 {i} 个嵌入输入超出范围！")
                embedding_inputs[i] = torch.clamp(embedding_inputs[i], min=0, max=num_embeddings - 1)

        # 处理类别特征
        # embedding_outputs = [layer(input) for layer, input in zip(self.embedding_layers, embedding_inputs)]
          
        # # 处理数值特征
        # numeric_outputs = [F.relu(layer(input)) for layer, input in zip(self.numeric_layers, numeric_inputs)]

        # 合并所有特征
        deep_input = torch.cat(embedding_outputs + numeric_outputs, dim=1)
        output = self.deep_model(deep_input)
        return output

In [19]:
print(DNNModel(1, overview_data))

DNNModel(
  (numeric_layers): ModuleList(
    (0-35): 36 x Linear(in_features=1, out_features=4, bias=True)
  )
  (embedding_layers): ModuleList(
    (0): EmbeddingLayer(
      (embedding): Embedding(5, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (1): EmbeddingLayer(
      (embedding): Embedding(2, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (2): EmbeddingLayer(
      (embedding): Embedding(3, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (3-4): 2 x EmbeddingLayer(
      (embedding): Embedding(4, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (5): EmbeddingLayer(
      (embedding): Embedding(2, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (6): EmbeddingLayer(
      (embedding): Embedding(5, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (7): EmbeddingLayer(
      (embedding): Embedding(3, 2)
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (8): EmbeddingLayer(
      (embe

*** 
### Train

dataset的形式会影响到训练和forward的代码

In [20]:
class HousePriceDataset(Dataset):
    def __init__(self, data, labels):
        """
        初始化数据集
        :param data: 包含特征的数据 DataFrame
        :param labels: 对应的标签
        """
        self.data = data
        self.labels = labels

    def __len__(self):
        """
        返回数据集的样本数量
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        根据索引返回对应的样本和标签
        :param idx: 样本索引
        :return: 样本和标签
        """
        # 提取特征
        catogorical_feature =['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
        numerical_feature = ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
        # 提取分类特征
        categorical_values = self.data[catogorical_feature].iloc[idx].values
        categorical_features = torch.tensor(categorical_values, dtype=torch.long)

        # 提取数值特征
        numerical_values = self.data[numerical_feature].iloc[idx].values
        numerical_features = torch.tensor(numerical_values, dtype=torch.float32)

        # 拼接特征
        features = torch.cat([categorical_features, numerical_features])

        # 获取标签
        if self.labels is not None:
            label = torch.tensor(self.labels.iloc[idx], dtype=torch.float32)
            return features, label
        return features

house_dataset = HousePriceDataset(preprocess_features(overview_data), overview_data['SalePrice'])
sample_features, sample_label = house_dataset[1]
print("样本特征:", sample_features)
cat = sample_features[:len(catogorical_feature)]
num = sample_features[len(catogorical_feature):]
print("样本分类特征:", cat, cat.shape)
print("样本数值特征:", num,num.shape)
print("样本标签:", sample_label)

样本特征: tensor([ 3.0000e+00,  1.0000e+00,  2.0000e+00,  3.0000e+00,  3.0000e+00,
         0.0000e+00,  2.0000e+00,  0.0000e+00,  2.4000e+01,  1.0000e+00,
         2.0000e+00,  0.0000e+00,  2.0000e+00,  1.0000e+00,  1.0000e+00,
         8.0000e+00,  8.0000e+00,  3.0000e+00,  3.0000e+00,  4.0000e+00,
         1.0000e+00,  2.0000e+00,  3.0000e+00,  1.0000e+00,  0.0000e+00,
         5.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  4.0000e+00,
         3.0000e+00,  6.0000e+00,  4.0000e+00,  1.0000e+00,  1.0000e+00,
         4.0000e+00,  4.0000e+00,  2.0000e+00,  3.0000e+00,  4.0000e+00,
         4.0000e+00,  8.0000e+00,  4.0000e+00, -1.1252e+00,  5.6371e-01,
         1.1341e-01,  2.7742e-02,  1.9482e+00,  1.6344e-01, -4.2419e-01,
        -8.1596e-01,  8.8826e-01, -3.5534e-01,  2.2183e-03,  3.4111e-01,
         4.1848e-01, -8.7012e-01, -1.3360e-01, -3.8197e-01, -8.3323e-01,
         4.0215e+00,  8.0995e-01, -7.6964e-01,  2.5389e-01, -2.0761e-01,
        -2.2644e-01,  7.4336e-01, -9.8044e-02

In [21]:
def train_model(model, train_loader,eval_loader, model_type='dnn', 
                epochs=400, batch_size=1024, learning_rate=0.001, patience=10):
    """
    Train model with early stopping and learning rate scheduling
    """
    # Setup device and criterion
    model.to('cpu')
    criterion = nn.HuberLoss(delta=1.0)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    
    # Initialize training variables
    best_val_loss = float('inf')
    no_improvement_count = 0
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                       factor=0.1, patience=5, min_lr=1e-6)
    train_losses, val_losses = [], []
    
    # Training loop
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = train_epoch(model, train_loader, optimizer, criterion, 
                               device, model_type)
        train_losses.append(train_loss)
        
        # Validation phase
        model.eval()
        val_loss = validate_epoch(model, eval_loader, criterion, device, model_type)
        val_losses.append(val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        # Learning rate scheduling
        lr_scheduler.step(val_loss)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement_count = 0
        else:
            no_improvement_count += 1
            if no_improvement_count >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    # Plot loss curves
    plot_losses(train_losses, val_losses)
    
    return model, train_losses, val_losses

def train_epoch(model, loader, optimizer, criterion, device, model_type):
    running_loss = 0.0
    for batch in loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        labels = labels.unsqueeze(1)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * labels.size(0)
    
    return running_loss / len(loader.dataset)

def validate_epoch(model, loader, criterion, device, model_type):
    running_loss = 0.0
    with torch.no_grad():
        for batch in loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs) 
            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
    
    return running_loss / len(loader.dataset)

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

def get_dataloader(x_train, x_eval, y_train, y_eval):
    train_dataset = HousePriceDataset(x_train, y_train)
    eval_dataset = HousePriceDataset(x_eval, y_eval)
    
    # 创建数据加载器
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)
    
    return train_loader, eval_loader

In [22]:
df = pd.read_csv(r'D:\github_repo_forked\lifetime_value\house-prices-advanced-regression-techniques\train.csv')
fill_missing_value(df)
df_processed = preprocess_features(df)


In [23]:
[df_processed[key].nunique() for key in catogorical_feature]

[5,
 2,
 3,
 4,
 4,
 2,
 5,
 3,
 25,
 9,
 8,
 5,
 8,
 6,
 8,
 15,
 16,
 4,
 4,
 5,
 6,
 5,
 5,
 5,
 7,
 7,
 6,
 5,
 2,
 6,
 4,
 7,
 6,
 7,
 4,
 6,
 6,
 3,
 4,
 5,
 5,
 9,
 6]

In [24]:
model = DNNModel(1, df_processed)


In [25]:
x_train, x_eval, y_train, y_eval = dnn_split(df_processed)
train_loader, eval_loader = get_dataloader(x_train, x_eval, y_train, y_eval)


In [27]:
model, train_losses, val_losses = train_model(model,train_loader , eval_loader,model_type='dnn', epochs=400, batch_size=256, learning_rate=0.001, patience=10)


IndexError: index out of range in self

In [62]:
for batch in train_loader:
    *features, labels = batch
    numeric_features = features[0]
    categorical_features = [f for f in features[1:]]
    print('num',numeric_features.size())
    print('cat',categorical_features[0].size())

num torch.Size([1024, 36])
cat torch.Size([1024])
num torch.Size([144, 36])
cat torch.Size([144])


In [ ]:
# 定义一个linear的模型
class LinearModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)
    
linear_model = LinearModel(79)


***
### Eval


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

def evaluate_regression(model, eval_loader, device):
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for numeric_input, *categorical_inputs, labels in eval_loader:
            numeric_input, labels = numeric_input.to(device), labels.to(device)
            categorical_inputs = [input.to(device) for input in categorical_inputs]
            outputs = model(numeric_input, *categorical_inputs)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    # Calculate metrics
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    
    # Print metrics
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # Plot predictions vs actuals
    plt.figure(figsize=(10, 6))
    plt.scatter(actuals, predictions, alpha=0.5)
    plt.plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], 'r--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Predictions vs Actuals')
    plt.show()
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

In [ ]:
evaluate_regression(model, eval_loader, device)